In [1]:
import pandas as pd
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
import random

In [2]:
import gc

In [3]:
from Config import *
import TextDataFrame as tdf
import utils
import Embedding as eb

import imp
imp.reload(tdf)
imp.reload(utils)
imp.reload(eb)

<module 'Embedding' from '/home/adrian/Projects/Competition/kaggle-toxic-comments-2021/Embedding.py'>

In [4]:
utils.set_seed()

In [5]:
df_processor = tdf.DatasetProcessor(DEFAULT_CLEAN_PROCEDURE)

### Read Data

In [6]:
weights_target_dict = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5,  'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}
text_col = 'comment_text'

df_train = pd.read_csv(os.path.join(DATA_PATH,"toxic-comment-classification-challenge/train.csv"))
df_train['y'] = df_processor.set_target(df_train, average_weights_dict = weights_target_dict)
df_train[text_col] = df_processor.clean_text(df_train[text_col])

  0%|          | 0/159571 [00:00<?, ?it/s]

AttributeError: 'UnboundLocalError' object has no attribute 'msg'

  0%|          | 0/159571 [00:00<?, ?it/s]

/home/adrian/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Wikipedia_talk:No_original_research/archive15#YouTube_art_as_primary_source" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/adrian/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/adrian/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Wikipedia:ELYES" looks like a URL. Beautiful Soup is not 

In [10]:
tokenizer = eb.Embedding()
tokenizer.fit(df_train[text_col])

In [11]:
df_train_new = utils.sample_binary(df_train, 'y')
y = df_train_new['y']

In [12]:
X_TFIDF = tokenizer.transform('TFIDF', df_train_new[text_col])

In [ ]:
X_BERT = tokenizer.transform('BERT', df_train_new[text_col], 100)

  0%|          | 0/325 [00:00<?, ?it/s]

In [4]:
import re
re.sub('\s+',' ', 'aa   bb')

'aa bb'

In [37]:
from sklearn.linear_model import Ridge

model_ridge_BERT = Ridge(alpha=0.5)
model_ridge_TFIDF = Ridge(alpha=0.5)

model_ridge_TFIDF.fit(X_TFIDF, y)
model_ridge_BERT.fit(X_BERT, y)

Ridge(alpha=0.5)

In [43]:
def predict(text_series, text_cleaner_func, tokenizer_transform_func, predict_func,
           ):
    cleaned_text = text_cleaner_func(text_series)
    cleaned_text = tokenizer_transform_func(cleaned_text)
    probability =  predict_func(cleaned_text)
    return probability
def evaluate(text_cleaner_func, tokenizer_transform_func, predict_func):

    df_val = pd.read_csv(os.path.join(DATA_PATH,"toxic-severity-rating/validation_data.csv"))

    p1 = predict(df_val['less_toxic'], 
                 text_cleaner_func, 
                 tokenizer_transform_func,
                 predict_func
                )

    p2 = predict(df_val['more_toxic'], 
                 text_cleaner_func, 
                 tokenizer_transform_func,
                 predict_func
                )

    return {"% Correct":(p1<p2).mean(), "% Equal":(p1==p2).mean()}

In [45]:
df_val = pd.read_csv(os.path.join(DATA_PATH,"toxic-severity-rating/validation_data.csv"))

In [48]:
text_series = df_val['less_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = tokenizer.transform('TFIDF', cleaned_text)
p1 = model_ridge_TFIDF.predict(cleaned_text)

text_series = df_val['more_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = tokenizer.transform('TFIDF', cleaned_text)
p2 = model_ridge_TFIDF.predict(cleaned_text)

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

In [50]:
{"% Correct":(p1<p2).mean(), "% Equal":(p1==p2).mean()}

{'% Correct': 0.6691576989504451, '% Equal': 0.0}

In [84]:
text_series = df_val['less_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = batch_transform(tokenizer, 'BERT', cleaned_text, 100)
p1_BERT = model_ridge_BERT.predict(cleaned_text)

text_series = df_val['more_toxic']
cleaned_text =  df_processor.clean_text(text_series)
cleaned_text = batch_transform(tokenizer, 'BERT', cleaned_text, 100)
p2_BERT = model_ridge_BERT.predict(cleaned_text)

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/302 [00:00<?, ?it/s]

In [86]:
{"% Correct":(p1_BERT<p2_BERT).mean(), "% Equal":(p1_BERT==p2_BERT).mean()}

{'% Correct': 0.6455759266640095, '% Equal': 0.0}